In [ ]:
from skimage.io import imread
import sys
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.applications import resnet50

In [ ]:
#将数据集写入文本中
label=os.listdir(train_path)
train_path='/kaggle/input/10-monkey-species/training/training/'
valid_path='/kaggle/input/10-monkey-species/validation/validation/'
with open('train.txt','w') as f:
    for l in range(len(label)):
        for filename in os.listdir(os.path.join(train_path,label[l])):
            line=f'{label[l]}/{filename} {l}\n'
            f.write(line)
with open('valid.txt','w') as f:
    for l in range(len(label)):
        for filename in os.listdir(os.path.join(valid_path,label[l])):
            line=f'{label[l]}/{filename} {l}\n'
            f.write(line)#重写数据类，可以将类名加入，也可以不加

In [ ]:
#计算类权重
v0=1087
v1=2189
v2=2386
v3=13158
v4=2577
total=sum([v0,v1,v2,v3,v4])
class_weight={0:(1/v0)*(total/5), 1:(1/v1)*(total/5), 2:(1/v2)*(total/5), 3:(1/v3)*(total/5), 4:(1/v4)*(total/5)}
class_weight


In [6]:
#数据增益方法
def augment_list():  # 16 oeprations and their ranges
    l = [ (AutoContrast, 0, 1),(Equalize, 0, 1),
        (Invert, 0, 1),(Rotate, 0, 30),(Posterize, 0, 4),
        (Solarize, 0, 256),(SolarizeAdd, 0, 110),(Contrast, 0.1, 1.9),
        (ShearX, 0., 0.3),(ShearY, 0., 0.3),(CutoutAbs, 0, 40),
        (TranslateXabs, 0., 100),(TranslateYabs, 0., 100),]
    return l
class RandAugment:
    def __init__(self, n, m):
        self.n = n
        self.m = m      # [0, 30]
        self.augment_list = augment_list()

    def __call__(self, img):
        ops = random.choices(self.augment_list, k=self.n)
        for op, minval, maxval in ops:
            val = (float(self.m) / 30) * float(maxval - minval) + minval
            img = op(img, val)
        return img
    
def mixup_data(x,y,alpha=0.9):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
   
    batch_size=x.shape[0]

    #index = tf.constant(torch.randperm(batch_size))
    index=  tf.random.shuffle(tf.range(batch_size))

    mixed_x = lam * x + (1 - lam) * tf.gather(x,index)
    y_a, y_b = y,tf.gather(y,index)
    return mixed_x, y_a, y_b, lam


In [ ]:
#制作数据集
class Dataset(object):
    
    CLASSES=None
    
    def __init__(self, 
                ann_file,
                classes=None,
                data_root=None,
                test_mode=False,
                batch_size=64):
        super(Dataset,self).__init__()
        
        self.ann_file=ann_file
        self.batch_size=batch_size
        self.test_mode=test_mode

        self.data=tf.data.Dataset.from_tensor_slices(self.load_annotations())
        # self.data=tf.data.TFRecordDataset('images.tfrecords')
        self.data=self.prepare(self.data,mode=self.test_mode)
        self.CLASSES = self.get_classes(classes)
        
    
    def load_annotations(self):
        #载入所有数据集到列表中
        assert isinstance(self.ann_file, str)
        data_filename = []
        data_label=[]
        
        #读取txt文件
        with open(self.ann_file) as f:
            samples = [x.strip().split(' ') for x in f.readlines()]
            for filename, gt_label in samples:
                data_filename.append(self.data_prefix+filename)
                data_label.append(int(gt_label))
            return data_filename,data_label
        
    #如果要用TFrecode格式的文件，需要将解析函数放入类中
    def prepare(self,ds,mode):
        AUTOTUNE = tf.data.experimental.AUTOTUNE
        ds = ds.map(self._parse_img,num_parallel_calls=AUTOTUNE)#简单的解析出图片
   
        if not mode:
            # num_parallel_calls=tf.data.AUTOTUNE 并行
            #cache可以缓存一些打开过的文件
            ds = ds.shuffle(1000).map(self._augment_func,
                        num_parallel_calls=tf.data.experimental.AUTOTUNE).cache()
        else:
            ds = ds.map(self._valid_func,
                       num_parallel_calls=tf.data.experimental.AUTOTUNE).cache()
        ds = ds.batch(self.batch_size)
        return ds.prefetch(buffer_size=AUTOTUNE)#训练的时候也读取数据
    
    def _parse_img(self,filename,label):

        file=tf.io.read_file(filename)
        img_raw=tf.image.decode_jpeg(file)
        
        return img_raw,label
        
    @tf.function
    def _augment_func(self,img,label):
        #以图的方式进行，更加快
        img = tf.image.resize(img, [256, 256])
        #batch_num=tf.shape(img)[0]
        #[batch_num,224,224,3]如果已经切batch了，就要把第一个维度加上去
        img = tf.image.random_crop(img,[224,224,3])
 
        img = tf.image.random_flip_left_right(img)
        return img,label
    
    
    @tf.function
    def _valid_func(self,img,label):
        
        img = tf.image.resize(img, [256, 256])/255.0
        img=tf.image.resize_with_crop_or_pad(img,224,224)
        return img,label    


    @classmethod
    def get_classes(cls, classes=None):
        if classes is None:
            return cls.CLASSES

        if isinstance(classes, (tuple, list)):
            class_names = classes
        else:
            raise ValueError(f'Unsupported type {type(classes)} of classes.')

        return class_names

In [ ]:
traindata=Dataset('/kaggle/working/train.txt',
                  '/kaggle/input/10-monkey-species/training/training/'
                 ,batch_size=128)
validdata=Dataset('/kaggle/working/valid.txt',
                  '/kaggle/input/10-monkey-species/validation/validation/')

In [ ]:
#GPU
stragy=tf.distribute.MirroredStrategy()

In [ ]:
batch_size=64
epochs=20
class MyModel(tf.keras.Model):
    
    def train_step(self, data):
        x,y = data
        
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            #y_pred=self(mixed_x,training=True)
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}


In [ ]:
with stragy.scope():
    
    model=resnet50.ResNet50(weights='imagenet')
    feature=keras.Model(model.input,model.layers[-3].output)
    inputs=layers.Input(shape=(224,224,3))
    x=feature(inputs)
    x=layers.GlobalAvgPool2D()(x)
    x=layers.Dropout(0.5)(x)
    outputs=layers.Dense(10)(x)

    mymodel=MyModel(inputs,outputs)

model.summary()

In [ ]:
lr_schedule=tf.keras.optimizers.schedules.ExponentialDecay(0.01,decay_steps=10000,decay_rate=0.96)


mymodel.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    optimizer='adam',
           #optimizer=tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True),
                    run_eagerly=True)

history=mymodel.fit(traindata.data,epochs=epochs,class_weight=class_weight)



In [ ]:
model.save('mymodel')

# TFRecord文件生成

In [1]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.

def _bytes_feature(value):
        """Returns a bytes_list from a string / byte."""
        if isinstance(value, type(tf.constant(0))):
            value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


#这个是不管输入是标量还是列表，统一用列表处理
def int64_feature(value):
    """Wrapper for inserting int64 features into Example proto.
    """
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def float_feature(value):
    """Wrapper for inserting float features into Example proto.
    """
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def bytes_feature(value):
    """Wrapper for inserting bytes features into Example proto.
    """
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

In [2]:
def _process_image(name,
                   img_path='/kaggle/input/coco128/coco128/images/train2017/',
                   ann_path='/kaggle/input/coco128/coco128/labels/train2017/'):
    # Read the image file.
    filename = img_path +name
    image_data = tf.io.read_file(filename)

    
    
    # Read the XML annotation file.
    filename = os.path.join(ann_path,name.split('.')[0]+'.txt')
    with open(filename,'r') as f:
        labels = []
        bboxes = []
        for line in f.readlines():
            
            line=line.strip().split()
            label = int(line[0])
            x1,y1,x2,y2 = float(line[1]), float(line[2]), float(line[3]), float(line[4])
            labels.append(label)
            bboxes.append((x1,y1,x2,y2))
    return image_data,labels,bboxes

In [3]:
def _convert_to_example(image_data,labels,bboxes):
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    for b in bboxes:
        assert len(b) == 4
        # pylint: disable=expression-not-assigned
        [l.append(point) for l, point in zip([ymin, xmin, ymax, xmax], b)]
        # pylint: enable=expression-not-assigned

    image_format = b'JPEG'
    example = tf.train.Example(features=tf.train.Features(feature={
            #'image/height': int64_feature(shape[0]),
            #'image/width': int64_feature(shape[1]),
            #'image/channels': int64_feature(shape[2]),
            #'image/shape': int64_feature(shape),
            'image/object/bbox/xmin': float_feature(xmin),
            'image/object/bbox/xmax': float_feature(xmax),
            'image/object/bbox/ymin': float_feature(ymin),
            'image/object/bbox/ymax': float_feature(ymax),
            'image/object/bbox/label': int64_feature(labels),
            #'image/object/bbox/label_text': bytes_feature(labels_text),
            #'image/object/bbox/difficult': int64_feature(difficult),
            #'image/object/bbox/truncated': int64_feature(truncated),
            #'image/format': bytes_feature(image_format),
            'image/encoded': _bytes_feature(image_data)#这是个张量字符串
            }))
    return example

In [4]:
def _add_to_tfrecord(name, tfrecord_writer):

    image_data,labels , bboxes= _process_image(name)
    example = _convert_to_example(image_data, labels, bboxes)
    tfrecord_writer.write(example.SerializeToString())

In [ ]:
def _get_output_filename(output_dir, name, idx):
    return '%s/%s_%03d.tfrecord' % (output_dir, name, idx)

In [ ]:
def run(dataset_dir, output_dir, name='train', shuffling=False):

    filenames = sorted(os.listdir(dataset_dir))
    if shuffling:
        random.seed(RANDOM_SEED)
        random.shuffle(filenames)

    # Process dataset files.
    i = 0
    fidx = 0
    while i < len(filenames):
        # Open new TFRecord file.
        tf_filename = _get_output_filename(output_dir, name, fidx)
   
        with tf.io.TFRecordWriter(tf_filename) as tfrecord_writer:
            j = 0
            while i < len(filenames) and j < 20:
                sys.stdout.write('\r>> Converting image %d/%d' % (i+1, len(filenames)))
                sys.stdout.flush()
                filename = filenames[i]
                
                _add_to_tfrecord(filename, tfrecord_writer)
                i += 1
                j += 1
            fidx += 1

    # Finally, write the labels file:
    # labels_to_class_names = dict(zip(range(len(_CLASS_NAMES)), _CLASS_NAMES))
    # dataset_utils.write_label_file(labels_to_class_names, dataset_dir)
    print('\nFinished converting the Pascal VOC dataset!')

In [ ]:
run('/kaggle/input/coco128/coco128/images/train2017/','/kaggle/working')

In [ ]:
# tf.Example 消息（或 protobuf）是一种灵活的消息类型，表示 {"string": value} 映射。它专为 TensorFlow 而设计，并被用于 TFX 等高级 API。

#要处理非标量特征，最简单的方法是使用 tf.io.serialize_tensor 将张量转换为二进制字符串。
#在 TensorFlow 中，字符串是标量。使用 tf.io.parse_tensor 可将二进制字符串转换回张量。

#反向转化为Example
#example_proto = tf.train.Example.FromString(serialized_example)


#读取
raw_image_dataset = tf.data.TFRecordDataset(filename)
# Create a dictionary describing the features.
image_feature_description = {
        'image/object/bbox/xmin': tf.io.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/label': tf.io.VarLenFeature(dtype=tf.int64),
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)



In [ ]:
#显示图片的方式，TENSORFLOW读入的是二进制图片
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
for image_features in parsed_image_dataset.take(1):
    image_raw = image_features['image/encoded'].numpy()
    #print(image_raw)
    p_img = Image.open(BytesIO(image_raw)) #这样可以保证图片是PIL的
    plt.imshow(p_img)

    
    

# 增益图片



In [ ]:
#将增益的compose pipeline 写成层的形式
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])

data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

#还是用map函数进行增益吧



In [ ]:
def parse_img(d):
    img = d['image/encoded']
    img= tf.image.decode_jpeg(img)
    img = tf.image.adjust_brightness(img, 0.4)
    d['image/encoded'] =img
    return d

In [2]:
##构造类，调用时什么都不用给
class Dataset(object):
    def __init__(self,
                tf_path=None,
                mode='train',
                batch_size=10):
        super(Dataset,self).__init__()
        
        self.batch_size=batch_size
        self.mode=mode
        #指定tf文件的名称即可
        self.data=tf.data.TFRecordDataset(tf_path)
        
        self.data=self.prepare(self.data,mode=self.mode)
        
    def prepare(self,ds,mode):
        AUTOTUNE = tf.data.experimental.AUTOTUNE
        ds = ds.map(self._parse_image_function)
        ds = ds.map(self._parse_img,num_parallel_calls=AUTOTUNE)#简单的解析出图片
   
        if mode=='train':
            ds = ds.shuffle(1000).map(self._augment_func,num_parallel_calls=tf.data.experimental.AUTOTUNE)
        else:
            ds = ds.map(self._valid_func,num_parallel_calls=tf.data.experimental.AUTOTUNE)
        ds = ds.batch(self.batch_size)
        return ds.prefetch(buffer_size=AUTOTUNE)
    
    def _parse_img(self,items):
        img_raw = tf.image.decode_jpeg(items['image/encoded'])
        items['image/encoded'] = img_raw
        return items
    
    def _parse_image_function(self,example_proto):
      # Parse the input tf.Example proto using the dictionary above.
      return tf.io.parse_single_example(example_proto, image_feature_description)
    
    @tf.function
    def _augment_func(self,items):#不能修改，可以复制
        new_item = items.copy()
        img = tf.image.resize(items['image/encoded'], [256, 256])

        img = tf.image.random_crop(img,[224,224,3])
 
        img = tf.image.random_flip_left_right(img)
        new_item['image/encoded'] =img
        return new_item
    
    @tf.function
    def _valid_func(self,img,label):
        img = tf.image.resize(img, [256, 256])/255.0
        img=tf.image.resize_with_crop_or_pad(img,224,224)
        return img,label
    
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 73)

In [1]:
#混合精度使用方法
#创建dtype策略，这个是全局的设置
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

#自定义模型，每一层都是用到全局设置的值
inputs = keras.Input(shape=(784,), name='digits')
dense2 = layers.Dense(num_units, activation='relu', name='dense_2')
x = dense2(x)

#为了保证稳定性，需要对模型最后一层用float32的设置，dtype='float32'，不一定是激活层
outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)


#将输入数据从 int8 强制转换为 float32
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255
#自定义训练
#这种计算需要考虑下溢的问题，损失放大就是一个防止出现下溢的技巧
loss_scale = policy.loss_scale
print('Loss scale: %s' % loss_scale)
#先设置优化器
optimizer = keras.optimizers.RMSprop()
optimizer = mixed_precision.LossScaleOptimizer(optimizer, loss_scale='dynamic')

#修改自定义训练方式
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        predictions = model(x)
        loss = loss_object(y, predictions)
        scaled_loss = optimizer.get_scaled_loss(loss)
    scaled_gradients = tape.gradient(scaled_loss, model.trainable_variables)
    gradients = optimizer.get_unscaled_gradients(scaled_gradients)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

@tf.function
def test_step(x):
    return model(x, training=False)



NameError: name 'mixed_precision' is not defined

# Apex

In [ ]:
#FP16的高效计算
!git clone https://github.com/NVIDIA/apex
%cd apex
!python setup.py install

#混合精度使用方法
#创建dtype策略，这个是全局的设置
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

#自定义模型，每一层都是用到全局设置的值
inputs = keras.Input(shape=(784,), name='digits')
dense2 = layers.Dense(num_units, activation='relu', name='dense_2')
x = dense2(x)

#为了保证稳定性，需要对模型最后一层用float32的设置，dtype='float32'，不一定是激活层
outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)


#将输入数据从 int8 强制转换为 float32
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255
#自定义训练
#这种计算需要考虑下溢的问题，损失放大就是一个防止出现下溢的技巧
loss_scale = policy.loss_scale
print('Loss scale: %s' % loss_scale)
#先设置优化器
optimizer = keras.optimizers.RMSprop()
optimizer = mixed_precision.LossScaleOptimizer(optimizer, loss_scale='dynamic')

#修改自定义训练方式
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        predictions = model(x)
        loss = loss_object(y, predictions)
        scaled_loss = optimizer.get_scaled_loss(loss)
    scaled_gradients = tape.gradient(scaled_loss, model.trainable_variables)
    gradients = optimizer.get_unscaled_gradients(scaled_gradients)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

@tf.function
def test_step(x):
    return model(x, training=False)